In [ ]:
# siplne angent

In [94]:
import os
from pathlib import Path
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List

In [95]:
# bing integrtion

In [96]:
from langchain_community.utilities import BingSearchAPIWrapper

In [97]:
# load env and vriables
load_dotenv()


True

In [98]:
search = BingSearchAPIWrapper()

In [99]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import requests
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

In [100]:
#gent tool

In [101]:
extraction_template = PromptTemplate(
    input_variables=["content", "person_name"],
    template="""
    Your task is to extract the following information about {person_name} from the {content}.
    Extracted information should be:
    - Name
    - Location
    - Occupation
    - General Info about the {person_name}
    
    Also extraxt name and infomration about persons , companies related to the {person_name} from the {content}
    """
)

summary_template = PromptTemplate(
    input_variables=["information_list", "person_name"],
    template="""
    Based on the following extracted information about {person_name}, provide a summary and conclusion:

    Information: {information_list}

    Summary and Conclusion:
    """
)

In [102]:
#chains definition

In [103]:
extraction_chain = LLMChain(llm=llm, prompt=extraction_template)
summary_chain = LLMChain(llm=llm, prompt=summary_template)

In [104]:
# bing model

In [105]:
class BingResult(BaseModel):
    title: str
    link: str
    
class BingResults(BaseModel):
    results: list[BingResult]

    def links(self):
        return [result.link for result in self.results]

In [106]:
def load_documents(loader_clc, urls):
    loader = loader_clc(urls)
    return loader.load()

def serch_bing(query, n_results):
    return BingResults(results=search.results(query, n_results))

In [107]:
# Function to extract information
def extract_information(chunk, person_name):
    return extraction_chain.run({"content": chunk.page_content, "person_name": person_name})

# Function to summarize information
def summarize_information(information_list, person_name):
    return summary_chain.run({"information_list": information_list, "person_name": person_name})

In [108]:
def get_info(person_name, n_results):
    #get results
    results = serch_bing(person_name, n_results)
    documents = load_documents(WebBaseLoader, results.links())
    # spliting text into the chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
    chunks = text_splitter.split_documents(documents)
    
    # Extract information from each chunk
    extracted_information = [extract_information(chunk, person_name) for chunk in chunks]
    
    # Combine extracted information into a single string
    information_list = "\n".join(extracted_information)
    
    # Generate summary
    summary_result = summarize_information(information_list, person_name)
    
    print("Summary and Conclusion:")
    print(summary_result)

In [109]:
get_info("Dimitry Sytyi", 5)

Summary and Conclusion:
### Summary and Conclusion

Based on the extracted information, here is a comprehensive summary of Dimitry Sytyi and related entities:

### Summary

#### Dimitry Sytyi
- **Name:** Dimitry Sytyi
- **Location:** Bangui, Central African Republic (CAR)
- **Occupation:** Leader at Wagner Group in Bangui
- **General Info:**
  - Dimitry Sytyi has risen to a prominent leadership position within the Wagner Group in Bangui, Central African Republic.
  - He is associated with the group's 'civilian activities' and holds significant influence in both business and security domains.
  - Despite facing challenges, such as the death of his mentor Yevgeny Prigozhin, Sytyi continues to maintain his role and influence within the organization.

#### Related Persons and Entities

1. **Yevgeny Prigozhin**
   - **Role:** Founder of the Wagner Group
   - **Relation to Sytyi:** Mentor and significant figure in Sytyi's rise within Wagner
   - **General Info:** Known for his close ties to 